In [ ]:
import torch, time
import torch.nn as nn
import torch.optim as optim

from src.utils import *
from src.override_resnet import *


class Args:
    arch = 50
    dataset = "ImageNet"
    # dataset = "CIFAR100"
    lr = 0.001
    momentum = 0.9
    batch = 16
    epochs = 10
    save_every = 1
    quan = "static"
    only_eval = True
    verbose = True


args = Args()

In [ ]:
def run_benchmark(model, img_loader):
    elapsed = 0
    model.eval()
    num_batches = 1
    # 이미지 배치들 이용하여 스크립트된 모델 실행
    for i, (images, target) in enumerate(img_loader):
        if i < num_batches:
            start = time.time()
            output = model(images)
            end = time.time()
            elapsed = elapsed + (end - start)
        else:
            break
    num_images = images.size()[0] * num_batches

    print("Elapsed time: %3.0f ms" % (elapsed / num_images * 1000))
    return elapsed

In [ ]:
def fuse_model(model) -> nn.Module:
    flag = False
    for m in model.modules():
        if m.__class__.__name__ == ResNet_quan.__name__:
            if flag == True:
                raise ValueError("ResNet_quan is already fused")
            flag = True
            torch.quantization.fuse_modules(
                m,
                ["conv1", "bn1", "relu"],
                inplace=True,
            )

        if type(m) == BottleNeck_quan:
            torch.quantization.fuse_modules(
                m,
                [
                    ["conv1", "bn1", "relu1"],
                    ["conv2", "bn2", "relu2"],
                    ["conv3", "bn3"],
                ],
                inplace=True,
            )
            if m.downsample is not None:
                torch.quantization.fuse_modules(
                    m.downsample,
                    ["0", "1"],
                    inplace=True,
                )
    return model

In [ ]:
# %% my code

args = Args()
# %% Load the ResNet-50 model
if args.quan == "fp32":
    # case 0 : no quantization case
    print("----------No quantization enabled")
    device = str(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
    model = layers_mapping[args.arch](
        weights=pretrained_weights_mapping[args.arch]
    ).to(device)

elif args.quan == "dynamic":
    # case 1 : Dynamic Quantization
    print("----------Dynamic Quantization enabled")
    device = "cuda"
    model = resnet50_quan(weights=pretrained_weights_mapping[args.arch]).to(device)
    quantized_model = torch.quantization.quantize_dynamic(
        model, {torch.nn.Linear}, dtype=torch.qint8
    )
    model = quantized_model

elif args.quan == "static":
    # case 2 : Static Quantization
    print("----------Static Quantization enabled")
    device = "cpu"
    model = resnet50_quan(weights=pretrained_weights_mapping[args.arch]).to(device)

elif args.quan == "qat":
    # case 3 : Quantization Aware Training
    print("----------Quantization Aware Training enabled")
else:
    raise ValueError("Invalid quantization method")

_folder_path = f"resnet{args.arch}_{args.dataset}" + "_" + args.quan
_file_name = (
    f"resnet{args.arch}_{args.dataset}_epoch"  # resnet18_cifar10_epoch{epoch}.pth
)

# %%Set up training and evaluation processes
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
device = "cpu"
print(device)

In [ ]:
model.to(device)
model.eval()

train_loader, test_loader = GetDataset(
    dataset_name=args.dataset,
    device=device,
    root="data",
    batch_size=64,
    num_workers=8,
)

# 0. REF (acc@1 : 80.35%, 시간 및 크기 측정만)

In [ ]:
ref_runtime = run_benchmark(model, test_loader)
print_size_of_model(model)
print(model.layer1[0])

fuse 확인

In [ ]:
model = fuse_model(model)
print(print_size_of_model(model))
print(model.layer1[0])

# calibration (training set 한 바퀴 돌림)

In [ ]:
model.qconfig = torch.quantization.get_default_qconfig("x86")
print(model.qconfig)
torch.quantization.prepare(model, inplace=True)
print("Post Training Quantization Prepare: Inserting Observers")

In [ ]:
_, _ = SingleEpochEval(model, train_loader, criterion, device)
print("Post Training Quantization: Calibration done")

convert

In [ ]:
torch.quantization.convert(model, inplace=True)
print("Post Training Quantization: Convert done")

# Static quantization 완료

In [ ]:
quantized_runtime = run_benchmark(model, test_loader)

eval_loss, eval_acc = SingleEpochEval(model, test_loader, criterion, device)
print_size_of_model(model)
print(f"Eval Loss: {eval_loss:.4f}, Eval Acc: {eval_acc:.2f}%")
print("Post Training Quantization: Eval done")

In [ ]:
print(f"Quantized time: {quantized_runtime:.4f}ms, Ref time: {ref_runtime:.4f}ms")